In [3]:
import torch
from torch import nn
from GetLoader import GetLoader
from ModelEmbedding import ModelEmbedding
from MyDataset import MyDataset
from GetInit import GetInit
from TrainFunc import TrainFunc
from TextCNN import TextCNN

In [2]:
data_root = {
    "train_path": '../../data/train_torch.csv',
    "test_path": "../../data/test_a.csv",
    "sub_path": "../../data/test_a_sample_submit.csv",
    "w2v_path": "../../data/word2vec.bin"
}
config = GetInit(data_root)
model_embedding = ModelEmbedding(data_root["w2v_path"])

train_dataset = MyDataset(model_embedding,
                          corpus=config.x_train,
                          corpus_label=config.y_train,
                          with_label=True)
test_dataset = MyDataset(model_embedding,
                         corpus=config.x_test,
                         with_label=False)
loader = GetLoader(train_dataset, test_dataset)

GetInit Start!
GetInit End!
ModelEmbedding End!


In [7]:
# 建立model
model = TextCNN(vocab_size=model_embedding.dict_length, embedding_dim=300, output_size=14)
model.init_weights(model_embedding.embedding, is_static=False)
model = model.cuda()
criterion = nn.NLLLoss()
opt = torch.optim.Adam(model.parameters(), lr=1e-4)
# 开始训练
mytrain = TrainFunc(model, criterion, opt, loader.train_loader, loader.valid_loader, loader.test_loader)

GetLoader End


In [8]:
best_model = mytrain.train(25)

Train Epoch: 1 [0/180000 (0%)]	Loss: 2.728758
Train Epoch: 1 [12800/180000 (7%)]	Loss: 1.678077
Train Epoch: 1 [25600/180000 (14%)]	Loss: 1.124747
Train Epoch: 1 [38400/180000 (21%)]	Loss: 0.710934
Train Epoch: 1 [51200/180000 (28%)]	Loss: 0.556901
Train Epoch: 1 [64000/180000 (36%)]	Loss: 0.673313
Train Epoch: 1 [76800/180000 (43%)]	Loss: 0.571208
Train Epoch: 1 [89600/180000 (50%)]	Loss: 0.396836
Train Epoch: 1 [102400/180000 (57%)]	Loss: 0.322723
Train Epoch: 1 [115200/180000 (64%)]	Loss: 0.530926
Train Epoch: 1 [128000/180000 (71%)]	Loss: 0.397607
Train Epoch: 1 [140800/180000 (78%)]	Loss: 0.372316
Train Epoch: 1 [153600/180000 (85%)]	Loss: 0.415599
Train Epoch: 1 [166400/180000 (92%)]	Loss: 0.357137
Train Epoch: 1 [179200/180000 (100%)]	Loss: 0.208100
	Loss: 0.6457(train)	|	Acc: 81.9%(train)
	Loss: 0.3172(valid)	|	Acc: 90.2%(valid)
	Micro: 0.9022(valid)	|	Macro: 0.8612(valid)
Now_best:0.8612
Train Epoch: 2 [0/180000 (0%)]	Loss: 0.422291
Train Epoch: 2 [12800/180000 (7%)]	Loss: 0.1

KeyboardInterrupt: 

In [9]:
criterion = nn.NLLLoss()
opt = torch.optim.Adam(model.parameters(), lr=1e-4)
# 开始训练
mytrain = TrainFunc(model, criterion, opt, loader.train_loader, loader.valid_loader, loader.test_loader)
best_model = mytrain.train(10)

Train Epoch: 1 [0/180000 (0%)]	Loss: 0.058866
Train Epoch: 1 [12800/180000 (7%)]	Loss: 0.022962
Train Epoch: 1 [25600/180000 (14%)]	Loss: 0.015323
Train Epoch: 1 [38400/180000 (21%)]	Loss: 0.041388
Train Epoch: 1 [51200/180000 (28%)]	Loss: 0.021106
Train Epoch: 1 [64000/180000 (36%)]	Loss: 0.066051
Train Epoch: 1 [76800/180000 (43%)]	Loss: 0.038057
Train Epoch: 1 [89600/180000 (50%)]	Loss: 0.014788
Train Epoch: 1 [102400/180000 (57%)]	Loss: 0.029982
Train Epoch: 1 [115200/180000 (64%)]	Loss: 0.031499
Train Epoch: 1 [128000/180000 (71%)]	Loss: 0.068466
Train Epoch: 1 [140800/180000 (78%)]	Loss: 0.029149
Train Epoch: 1 [153600/180000 (85%)]	Loss: 0.062011
Train Epoch: 1 [166400/180000 (92%)]	Loss: 0.035421
Train Epoch: 1 [179200/180000 (100%)]	Loss: 0.012467
	Loss: 0.0342(train)	|	Acc: 99.4%(train)
	Loss: 0.1855(valid)	|	Acc: 94.4%(valid)
	Micro: 0.9438(valid)	|	Macro: 0.9352(valid)
Now_best:0.9352
Train Epoch: 2 [0/180000 (0%)]	Loss: 0.045769
Train Epoch: 2 [12800/180000 (7%)]	Loss: 0.0

KeyboardInterrupt: 

In [10]:
ans=mytrain.predict()

In [15]:
import pandas as pd
import numpy as np
df_sub=pd.read_csv(data_root["sub_path"])
df_sub.label=np.array(ans)
df_sub.label=df_sub.label.astype("int64")
save_name="./textcnn_{:.4f}.csv".format(mytrain.best_score)
df_sub.to_csv(save_name, index=False)

In [20]:
mytrain.best_score

0.9355598130314265